## Before we start building a model

Make sure you name a text file you want to use for training 'input.txt' and upload it to Colab. If the file name is different, modify the cell below.

In [1]:
from google.colab import files
uploaded = files.upload()


Saving text_clean.txt to text_clean (2).txt


In [2]:
# read it in to inspect it
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  798706


In [4]:
# let's look at the first 1000 characters
print(text[:1000])



DOKUMENT: 9443.pdf

VaFu06-T

List 1

Maximum a minimum funkcie

RNDr. Beáta Vavrinčíková

U: Slová maximálny a minimálny sú síce cudzieho pôvodu, ale v našej reči sa už bežne
používajú, napríklad maximálna povolená rýchlosť auta alebo minimálna mzda zamest-
nanca.

Ž: Alebo podnikateľ chce dosiahnuť maximálne zisky pri minimálnych nákladoch.

U: Áno, a v tom, ako to dosiahnuť, mu môže pomôcť aj matematika. Tieto dva pojmy –
maximum a minimum sa totiž týkajú aj funkcií. Spoločným názvom sa označujú tiež ako
extrémy, teda niečo, čo sa vymyká z priemeru. Ukážeme si to na príklade funkcie, ktorá
vyjadruje závislosť hodnoty amerického dolára od eura. Z internetu sme si stiahli graf,
ktorý zachytáva túto závislosť v období od augusta 2006 do júla 2007:

U: Vedel by si z grafu určiť, kedy mal dolár maximálnu hodnotu?

Ž: Pravdaže,


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#%()+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]`abcdefghijklmnopqrstuvxyz{|}¬±·ÁÚáäéíóôúüýČčĎďĺĽľŇňŕŠšťŽžαβπ–‘„•←→↔⇐⇑⇒⇔∀∃∅∈−∗√∞∧∨∪⊂◦ﬁﬂ
149


For now, we won’t perform proper tokenization (it will be your task to train a tokenizer and use it as part of the homework assignment). Instead, we’ll simply tokenize by character.

In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[66, 67, 67, 2, 78, 66, 63, 76, 63]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the first 100 characters of 1000 characters we looked at earier will to the GPT look like this

torch.Size([798706]) torch.int64
tensor([ 0,  0, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27,  0, 33, 44, 40, 50, 42, 34, 43,
        49, 24,  2, 23, 18, 18, 17, 12, 74, 62])


Let us use 90% for training and last 10% for validation.

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Blocks

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([ 0,  0, 27, 27, 27, 27, 27, 27, 27])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 0
when input is tensor([0, 0]) the target: 27
when input is tensor([ 0,  0, 27]) the target: 27
when input is tensor([ 0,  0, 27, 27]) the target: 27
when input is tensor([ 0,  0, 27, 27, 27]) the target: 27
when input is tensor([ 0,  0, 27, 27, 27, 27]) the target: 27
when input is tensor([ 0,  0, 27, 27, 27, 27, 27]) the target: 27
when input is tensor([ 0,  0, 27, 27, 27, 27, 27, 27]) the target: 27


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 19,  10,   2,  22,  19,  21,  10,   0],
        [102, 112,  63,  72,  92,   2,  73,   2],
        [  2,  74,  76,  63,   2,  74,  70,  59],
        [  2,  80,  82,  69,  73,  72,  92,  80]])
targets:
torch.Size([4, 8])
tensor([[ 10,   2,  22,  19,  21,  10,   0,  50],
        [112,  63,  72,  92,   2,  73,   2,  15],
        [ 74,  76,  63,   2,  74,  70,  59,  78],
        [ 80,  82,  69,  73,  72,  92,  80,  59]])
----
when input is [19] the target: 10
when input is [19, 10] the target: 2
when input is [19, 10, 2] the target: 22
when input is [19, 10, 2, 22] the target: 19
when input is [19, 10, 2, 22, 19] the target: 21
when input is [19, 10, 2, 22, 19, 21] the target: 10
when input is [19, 10, 2, 22, 19, 21, 10] the target: 0
when input is [19, 10, 2, 22, 19, 21, 10, 0] the target: 50
when input is [102] the target: 112
when input is [102, 112] the target: 63
when input is [102, 112, 63] the target: 72
when input is [102, 112, 63, 72] the target

In [12]:
print(xb) # our input to the model

tensor([[ 19,  10,   2,  22,  19,  21,  10,   0],
        [102, 112,  63,  72,  92,   2,  73,   2],
        [  2,  74,  76,  63,   2,  74,  70,  59],
        [  2,  80,  82,  69,  73,  72,  92,  80]])


## First model

Let’s build the simplest model we can think of.

Some notation:
* **B** — batch size (number of independent sequences processed in parallel)
* **T** — time dimension, number of tokens per sequence
* **C** — number of channels / classes = vocabulary size (the dimensionality of the logits for each token)

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            # reshaping because of `F.cross_entropy` definition
            logits = logits.view(B*T, C)  # shape (B*T, C)
            targets = targets.view(B*T)   # shape (B*T, )
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# this is before training!
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 149])
tensor(5.3691, grad_fn=<NllLossBackward0>)

ČBeDp,·←ý·αcq;mK∪,{iňjO:GMc⊂G0√∅Hr{/xĺ∈ČÚ(C∈á)4/∪a=6∃31|xí⇐Ui1↔∧xA∃jI→fŇŠü∞ňľó↔YX∧óX–-⊂ﬁfí7z0}⇔Ľ


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.4618120193481445


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ňÚ/„¬)é‘ ň;-T>∨!X{ﬂY`ľá}ý· Ú?I∈+↔?∞<np3∃pď#9#;q–-xd∧Á‘NČ∈`Ľý⇑Sβ·š|x0ď(ré:C¬<B-ŕĎYhL∞lé–j,n∞Č−!‘)pď·!o∈‘p√h∅íüJa%Lu-Bn<z8`ľŽﬁhc(∀c<xZm∅D2[∃◦fŠQĽ←k#ďĽ•U}f`Ľ⇐A`gŠoô(š∈QĽ=9<bJβt∅E3z◦OsšgCťá↔E∗0T:⊂rKIäp=ü?∨ťW-∞.9ršnα)DŇ4O)c⊂aP"e·Š√∅→Kúβ⊂RéCe↔go∀±CĎČG∞↔fĽﬂOh√x>ﬂ8FcŽĎ⇒∀5jhDc#s#x2šY}B{ÁF∪m∞⇒óβDc⊂1=éβ→íf0·AÚ•éňÚαŕô2GW∧V]⇔HňSJZgčôSﬁ"ť„sčB±
HyLe2ú◦sl"D/Fó=úôQαu→∀αdo⊂I∅⇒žT←7ÚVm⇔Klp√)>]⇑oZn"OjX7ŕď(á?mi◦OJ→Y>aC6zﬂ4ý[KlOs
U⊂ai
+s]„Z⇔A?`5Ú49gpULy6ý◦1jRb4bp2,Q<?ô∧ť→3/ĽE#W]7∀ﬁKCšfČoB‘tnŕj⇐U/š"D◦β8


## Let's improve it a bit

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3
max_iters = 1000
eval_interval = 100
eval_iters = 200

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            # reshaping because of `F.cross_entropy` definition
            logits = logits.view(B*T, C)  # shape (B*T, C)
            targets = targets.view(B*T)   # shape (B*T, )
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


0.022201 M parameters
step 0: train loss 5.5696, val loss 5.5364
step 100: train loss 5.4579, val loss 5.4182
step 200: train loss 5.3320, val loss 5.3010
step 300: train loss 5.2211, val loss 5.2071
step 400: train loss 5.1139, val loss 5.0929
step 500: train loss 5.0122, val loss 4.9818
step 600: train loss 4.9049, val loss 4.8907
step 700: train loss 4.8083, val loss 4.7812
step 800: train loss 4.7132, val loss 4.6878
step 900: train loss 4.6169, val loss 4.5942
step 999: train loss 4.5231, val loss 4.5063

iČ1−ﬂhoT∈ČT∀zL◦c·IIijéďh◦%∀Ď9ŽS⇐éhšTe0KNĎ!Úľ–ŕáhP=ü+nŽπĺÁ←↔š∞J
Qe,SEs`{v◦<ajZL◦∀◦⇒)`i¬4∈⊂G3 4IJ/Nk∗c←−>mesv0O2žužEf∈f⊂jno•←ôXĎ"!ÚľÚ=OoŽ◦Š↔0  rEuáúč9Z¬ĎóI⇒⇑N⇒∞x–ýN?.⇐ĺLk9=.dgZ64nπCπn5že∞Ďóš0äZF◦Uk)ťĽ2Č/¬hFS∅β ∃SUpUäW4ish?4){UQjúuj„W↔r·ď|,↔šakπäníž#áE→c`e∗•r lš{D6Ay0
CYSHsŠP−73αČYﬁĺUZcC{√F∧ŕm T
XeF(Ň⇑y√nši1⇑uR⇔+non–ŕ⇔5"}X!vI⇒√Cd±j·íd"!ŽSuľ(:í9y→U∧⇐tý−
√+TČhujaW[aÚ∞←/#j--Lπ?2: h,Ob∞Ň⇐√e⊂ﬂU;D→ťšľHOolﬁ>[2.ﬁóFcí.↔)t[69Č·j∪ú ∃ľĺKi>"Rβ◦
áM‘J∃∞b⇑óíČ„∃ôvH0bajšťL◦∀?.meť.?ke6óš(

## Normalization Layer

In [18]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

## The mathematical trick in self-attention

In [19]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [20]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [22]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [23]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [24]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [25]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [26]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [27]:
k.var()

tensor(1.0449)

In [28]:
q.var()

tensor(1.0700)

In [29]:
wei.var()

tensor(1.0918)

In [30]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [31]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [32]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(-0.1431), tensor(1.0705))

In [33]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(0.0073), tensor(1.0177))

## Transformer

You may want to refer directly to [the git repo](https://github.com/karpathy/ng-video-lecture) instead though.

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

# for saving the model
import os, json
SAVE_DIR = "./final_model"
MODEL_PATH = os.path.join(SAVE_DIR, "model_final.pt")
META_PATH = os.path.join(SAVE_DIR, "meta.json")
os.makedirs(SAVE_DIR, exist_ok=True)

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.220565 M parameters
step 0: train loss 5.1663, val loss 5.1654
step 100: train loss 2.9738, val loss 3.0047
step 200: train loss 2.7645, val loss 2.7604
step 300: train loss 2.6609, val loss 2.6836
step 400: train loss 2.5928, val loss 2.6219
step 500: train loss 2.5212, val loss 2.5564
step 600: train loss 2.4308, val loss 2.4578
step 700: train loss 2.3543, val loss 2.4251
step 800: train loss 2.2606, val loss 2.3114
step 900: train loss 2.2153, val loss 2.2712
step 1000: train loss 2.1485, val loss 2.2030
step 1100: train loss 2.0880, val loss 2.1651
step 1200: train loss 2.0343, val loss 2.1287
step 1300: train loss 2.0007, val loss 2.0750
step 1400: train loss 1.9568, val loss 2.0629
step 1500: train loss 1.9200, val loss 2.0061
step 1600: train loss 1.8923, val loss 1.9971
step 1700: train loss 1.8621, val loss 1.9682
step 1800: train loss 1.8382, val loss 1.9524
step 1900: train loss 1.8215, val loss 1.9213
step 2000: train loss 1.7758, val loss 1.8874
step 2100: train loss 1.

In [37]:
def save_meta():
    meta = {
        "block_size": block_size,
        "n_embd": n_embd,
        "n_head": n_head,
        "n_layer": n_layer,
        "dropout": dropout,
        "chars": chars,             # to rebuild vocab exactly
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

save_meta()
torch.save(model.state_dict(), MODEL_PATH)
print(f"Saved final model to {MODEL_PATH} and meta to {META_PATH}")

Saved final model to ./final_model/model_final.pt and meta to ./final_model/meta.json


## Using the saved model

The following code recreates the model and reads saved weights.

In [38]:
# - Assumes you trained & saved with:
#   - MODEL:  ./final_model/model_final.pt
#   - META:   ./final_model/meta.json

import torch
import torch.nn as nn
from torch.nn import functional as F
import os, json

SAVE_DIR = "./final_model"
MODEL_PATH = os.path.join(SAVE_DIR, "model_final.pt")
META_PATH = os.path.join(SAVE_DIR, "meta.json")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if not (os.path.exists(MODEL_PATH) and os.path.exists(META_PATH)):
    raise FileNotFoundError(f"Missing model or meta file in {SAVE_DIR}")

with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

block_size = meta["block_size"]
n_embd     = meta["n_embd"]
n_head     = meta["n_head"]
n_layer    = meta["n_layer"]
dropout    = meta["dropout"]
chars      = meta["chars"]

vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

# ---------- model definition ----------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd_):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd_, 4 * n_embd_),
            nn.ReLU(),
            nn.Linear(4 * n_embd_, n_embd_),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd_, n_head_):
        super().__init__()
        head_size = n_embd_ // n_head_
        self.sa = MultiHeadAttention(n_head_, head_size)
        self.ffwd = FeedFoward(n_embd_)
        self.ln1 = nn.LayerNorm(n_embd_)
        self.ln2 = nn.LayerNorm(n_embd_)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# ---------- load weights ----------
model = BigramLanguageModel().to(device)
state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.eval()

# ---------- generate ----------
seed_text = ""  # you can add a prompt here if you want
if seed_text:
    for c in seed_text:
        if c not in stoi:
            raise ValueError(f"Out-of-vocab char in seed: {repr(c)}")
    start = torch.tensor([encode(seed_text)], dtype=torch.long, device=device)
else:
    start = torch.zeros((1, 1), dtype=torch.long, device=device)

generated = model.generate(start, max_new_tokens=500)[0].tolist()
print(decode(generated))



pýrade je pravda právedzne mocninou.

U: No, alebo si o pusahuje dno šípku b.

KrAv07-12

List 10 9

U: Môžem byť 45, , dostanem

namenšie napríklade, teda alebo odmocné číslo, rebok: alebo nápak, výrazu jasný nemusí do bude riadiee

do niako liarené čísla, že

• je z voľkou majú zapiateľ príklad čísla pred t0 z nemať:ia predsum určiť.

y Tinie bolmoho výriar, predsum bolo veľmiee podiania položitia.

1
U: Zviju deﬁničný a oboru dobeﬁnícia premene, že pár. ENu ∈ D, dostaní 30 mocníme s riadmie
